## Introduction

Movie Theatre companies are somewhat easy to analyze. Taking a quick look at their earnings report, you'll see majority of their revenue are on Admission and Food & Bev. sales. And these are extremely dependent on the movies that are made and released. For example, with the huge success of Barbie and Oppenheimer in Q2 2023, AMC recorded their highest Admission revenue in over the past 3 years. In essence, if movies do well in theatres, then AMC does well. So in this series, we will be analyzing box office sales and seeing if AMC is worth investing in right before their earnings.   

In this part, you will see how we will download and clean the data using Python and SQL.

Luckily for us, box office numbers and AMC earnings reports are publicly available online. However, we cannot simply click a button to download all the data necessary, so we will write some code to obtain the data.

For our box office data we will be using the boxofficemojo.com website which posts daily sales data for each movie. For AMC revenue, we will be accessing their earnings reports posted on their website. Then we will be uploading all this data onto our SQL server.

## The Code

First and foremost, if you want to follow along and run through the code on your own, you will obviously need Python and SQL up and running. For this series, I will be using SQL Server.

Now, we will import the necessary packages to help us do the heavy lifting.

In [8]:
import pandas as pd
import lxml
import numpy as np
import time
from datetime import datetime
import re
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

#supress annoying warnings on Jupyter Lab - don't do this if you're still learning
import warnings
warnings.filterwarnings('ignore')


#necessary to connect to our SQL Server
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pyodbc

Once we have imported the necessary packages, we will now analyze the boxofficemojo website to understand how we will scrape the data. 

Going to the Calendar page we see the url change consistently depending on the month and year selected on the dropdown. For example, when I change the month and year to November 2022, we see the url change to boxofficemojo.com/calendar/2022-11-01/ and movie releases for that particular month. So this tells us we will need to create a list or a dictionary to loop through all the month and years. Also, when making our requests, we know our base string will be boxofficemojo.com/calendar/ while we make the latter end dynamic, changing only the date in the format of 'YYYY-mm-dd'.

Below is the dictionary I have made that we can loop through. I'm sure there's a way to make this on Python, but I decided to use Excel and just drag down on the cells to create this in less than a minute.

In [9]:
month_dict = {
    '2017-01-01':'Jan',
    '2017-02-01':'Feb',
    '2017-03-01':'Mar',
    '2017-04-01':'Apr',
    '2017-05-01':'May',
    '2017-06-01':'Jun',
    '2017-07-01':'Jul',
    '2017-08-01':'Aug',
    '2017-09-01':'Sep',
    '2017-10-01':'Oct',
    '2017-11-01':'Nov',
    '2017-12-01':'Dec',
    '2018-01-01':'Jan',
    '2018-02-01':'Feb',
    '2018-03-01':'Mar',
    '2018-04-01':'Apr',
    '2018-05-01':'May',
    '2018-06-01':'Jun',
    '2018-07-01':'Jul',
    '2018-08-01':'Aug',
    '2018-09-01':'Sep',
    '2018-10-01':'Oct',
    '2018-11-01':'Nov',
    '2018-12-01':'Dec',
             }

Next we will write a function to scrape all the movies released in that particular month and obtain the url to be able to access the daily box office sales for that particular movie. 

We know we will run a loop through our dictionary we created so for our first function we will need to pass two arguments, the url to access the page and the month for labelling purposes. The output of this function will be a dataframe containing the Title of the movie, the URL, Distributor and if it's a wide release or limited release movie.

See comments in the code for the justification of each line I write

In [10]:
def month_release_processing(url, month):
    'input url, returns dataframe'
    s1 = pd.read_html(url, extract_links='all') #reads the table present in the page and extracts the links from the table
    
    s2 = s1[0]
    s2.columns = ['Raw_Movie', 'Raw_Distributor','Raw_Scale'] #change column names to something more understandable

    #in this next block of code, when adding the argument extract_links='all', it actually returns a tuple in each column so we will split that into separate columns
    s2[['Movie','M_URL']] = pd.DataFrame(s2['Raw_Movie'].tolist())
    s2[['Distributor','Drop_1']] = pd.DataFrame(s2['Raw_Distributor'].tolist())
    s2[['Scale','Drop_2']] = pd.DataFrame(s2['Raw_Scale'].tolist())

    #creating a column for labelling purposes
    s2['Release_Month'] =  month 

    #list of columns to drop because they are either empty or we don't care about them
    drop_list = [
    'Raw_Movie',
    'Raw_Distributor',
    'Raw_Scale',
    'Drop_1',
    'Drop_2'
    ]
    s3 = s2.drop(drop_list, axis=1) #drop columns I don't care about

    s4 = s3[s3['M_URL'].str.contains("None")==False] #remove rows with no URLs (basically empty rows scraped)
    return s4

### Example Output of function

In [15]:
month_dict = {
    '2017-01-01':'Jan',
    '2017-02-01':'Feb'}

calend_url = 'https://www.boxofficemojo.com/calendar/{date}/'

for j, k in month_dict.items():
    test_url = calend_url.format(date=j) #creating the url
    output = month_release_processing(test_url, k)

output.head()

,Movie,M_URL,Distributor,Scale,Release_Month
0,Mr. Gaga: A True Story of Love and Dance Biog...,/release/rl1064666625/?ref_=bo_rs_table_1,Abramorama,Limited,Feb
1,The Lure Drama Fantasy Horror Musical Thr...,/release/rl285246977/?ref_=bo_rs_table_2,Janus Films,Limited,Feb
3,Irreplaceable Comedy Drama With: François C...,/release/rl2097645057/?ref_=bo_rs_table_3,Distrib Films,Limited,Feb
5,Rings Drama Horror Mystery Romance With: ...,/release/rl275547649/?ref_=bo_rs_table_4,Paramount Pictures,Wide,Feb
6,The Space Between Us Drama Romance Sci-Fi ...,/release/rl1216120321/?ref_=bo_rs_table_5,STX Entertainment,Wide,Feb


Here we see the output of our code. The M_URL column is an important one because we can now access the daily box office sales for that particular movie.

The next thing we have to do now is to loop through the movie urls. So we will create a dictionary to loop through the urls and label it. See comments in the code to understand justification for each line written.

In [17]:
def get_urls(df):
    'input dataframe returns list of dictionaries'
    url_list = df[['Movie','M_URL']].to_dict('records') #converts our output into a dictionary
    return url_list

### Example Output of function

In [19]:
get_urls(output)[0:5]

[{'Movie': 'Mr. Gaga: A True Story of Love and Dance  Biography  Documentary  With: Ohad Naharin, Tzofia Naharin, Eliav Naharin, Judith Brin Ingber1 hr 40 minCast, Crew, and Company Info',
  'M_URL': '/release/rl1064666625/?ref_=bo_rs_table_1'},
 {'Movie': 'The Lure  Drama  Fantasy  Horror  Musical  Thriller  With: Marta Mazurek, Michalina Olszanska, Kinga Preis, Andrzej Konopka1 hr 32 minCast, Crew, and Company Info',
  'M_URL': '/release/rl285246977/?ref_=bo_rs_table_2'},
 {'Movie': 'Irreplaceable  Comedy  Drama  With: François Cluzet, Marianne Denicourt, Christophe Odent, Patrick Descamps1 hr 42 minCast, Crew, and Company Info',
  'M_URL': '/release/rl2097645057/?ref_=bo_rs_table_3'},
 {'Movie': "Rings  Drama  Horror  Mystery  Romance  With: Matilda Anna Ingrid Lutz, Alex Roe, Johnny Galecki, Vincent D'Onofrio1 hr 42 minCast, Crew, and Company Info",
  'M_URL': '/release/rl275547649/?ref_=bo_rs_table_4'},
 {'Movie': 'The Space Between Us  Drama  Romance  Sci-Fi  With: Gary Oldman, A

Now that we are able to loop through each movie and access each page, we will now need to understand how each movie page is formatted.

When we inspect the individual movie page there are three different types of results:

1) Movie has an extremely limited release / only released online or DVD therefore has no data
2) Movie has an extremely limited release therefore data is only available by Weekend or Week
3) Movie has enough sales that daily data is available

The third result is what we want to scrape. While there are dollars associated with the second result, it is not significant enough to warrant writing customized code for to process as the format is different from the daily table.

Now that we have the url for each movie, we can now write some code to process each page and scrape the daily sales. See comments in the code to understand justification for each line written.

In [20]:
#this for labelling purposes and returns a cleaner movie name
def get_movie_title(url):
    """inside movie_rev_processing function
    input url return text"""
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html_page = urlopen(req).read()
    soup = BeautifulSoup(html_page, 'lxml')
    title = soup.find('h1', {"class":"a-size-extra-large"})
    return title.get_text()

def movie_rev_processing(url):
    """function inside a for loop
    input a url (string), return dataframe"""

    #added a try, except to ensure our code doesn't break
    try:
        s1 = pd.read_html(url, extract_links='all') #read url table and extract links to be able to parse the dates easily
        s2 = s1[0]

        #To catch movies with limited data (first or second result we described above)
        #we default to NA and Zeros as there is probably no data available anyways
        if len(s2.columns) < 11:
            date_string = '1900-01-01'
            data_ = {
                'Category':['Less 11', 'Less 11'],
                'DateString': ['NA', 'NA'],
                'Date': [date_string, date_string],
                'DOW': ['NA', 'NA'],
                'DailyDollars': [0.0, 0.0],
                'Theatres': [0.0, 0.0],
                'Month': ['NA', 'NA'],
                'Event': ['NA', 'NA']
            }
            df = pd.DataFrame(data = data_)
            df['Date'] = [datetime.strptime(date_, '%Y-%m-%d').date() for date_ in df['Date']]
            return df

        #just in case there is weird data structures going on that we did not notice when inspecting movie pages
        elif len(s2.columns) > 11:
            date_string = '1900-01-01'
            data_ = {
                'Category':['Greater 11', 'Greater 11'],
                'DateString': ['NA', 'NA'],
                'Date': [date_string, date_string],
                'DOW': ['NA', 'NA'],
                'DailyDollars': [0.0, 0.0],
                'Theatres': [0.0, 0.0],
                'Month': ['NA', 'NA'],
                'Event': ['NA', 'NA']
            }
            df = pd.DataFrame(data = data_)
            df['Date'] = [datetime.strptime(date_, '%Y-%m-%d').date() for date_ in df['Date']]
            return df

        #this is the data we really care about
        else:
            #we add this list because when pandas reads the table and extracts links, the column names are weird so we will rename them
            fix_cols = [
                'Date',
                'DOW',
                'Rank',
                'Daily',
                '+-YD',
                '+-LW',
                'Theatres',
                'Avg',
                'To Date',
                'Day',
                'Estimated'
            ]
        
            s2.columns = fix_cols #we rename the columns here

            #in this block of code we will split each column as most don't have links
            #we only care about the date column link because it shows the date in a cleaner format
            #the date column when extracted by itself (no links) combines the holiday/event and is difficult to parse through as it's not consistent
            #ex Christmas would be Dec25Christmas
            s2[['DateString','D_URL']] = pd.DataFrame(s2['Date'].tolist())
            s2[['DOW','Drop_1']] = pd.DataFrame(s2['DOW'].tolist())
            s2[['DailyDollars','Drop_2']] = pd.DataFrame(s2['Daily'].tolist())
            s2[['Theatres','Drop_3']] = pd.DataFrame(s2['Theatres'].tolist())

            #we parse through the date column and clean it up
            s2['P_Date'] = s2['D_URL'].apply(lambda st: st[st.find("date/")+5:st.find("/?ref")])

            #now we create a new dataframe with the columns we only care about
            s3 = s2[['DateString', 'P_Date', 'DOW', 'DailyDollars', 'Theatres']]
        
        
            #adding event/holiday columns        
            s3['Month'] = s3['DateString'].str[:4]
            s3['PreDay'] = s3['DateString'].str[4:]
            s3['DayCheck'] = [s[:2] if re.search('[A-Za-z]', s) != None else s[:2] for s in s3['PreDay']] #get day number
            s3['EventCheck'] = [re.search('[A-Za-z]', event) if re.search('[A-Za-z]', event) != None else "No Event" for event in s3['DayCheck']] #checks for alphabet in string
            s3['Event_start'] = [s.start() if s != "No Event" else "NA" for s in s3['EventCheck']] #gets starting position of non-number
            s3['Event'] = s3['PreDay'].str[2:] #gets event name

            #convert strings to float type to be able to do mathematical functions
            s3['DailyDollars'] = s3['DailyDollars'].replace('[\$,]', '', regex=True).astype(float)
            s3['Theatres'] = [float(str(i).replace(",", "")) for i in s3["Theatres"]]
            s3['Date'] = [datetime.strptime(date_, '%Y-%m-%d').date() for date_ in s3['P_Date']] #convert from string to date type

            #set to pass to be able to filter later on
            s3['Category'] = 'Pass'
        
            s4 = s3[['Category','DateString', 'Date', 'DOW', 'DailyDollars', 'Theatres', 'Month','Event']]
            
            return s4

    #catch just in case and won't break the code
    except:
        date_string = '1900-01-01'
        data_ = {
            'Category':['No Data', 'No Data'],
            'DateString': ['NA', 'NA'],
            'Date': [date_string, date_string],
            'DOW': ['NA', 'NA'],
            'DailyDollars': [0.0, 0.0],
            'Theatres': [0.0, 0.0],
            'Month': ['NA', 'NA'],
            'Event': ['NA', 'NA']
        }
        df = pd.DataFrame(data = data_)
        df['Date'] = [datetime.strptime(date_, '%Y-%m-%d').date() for date_ in df['Date']]
        return df
    


### Example output of code

In [24]:
ex_url = 'https://www.boxofficemojo.com/release/rl1077904129/?ref_=bo_tt_gr_1'
get_movie_title(ex_url)

'Barbie'

In [25]:
movie_rev_processing(ex_url).head()

,Category,DateString,Date,DOW,DailyDollars,Theatres,Month,Event
0,Pass,Jul 21,2023-07-21,Friday,70503178.0,4243.0,Jul,
1,Pass,Jul 22,2023-07-22,Saturday,47812356.0,4243.0,Jul,
2,Pass,Jul 23,2023-07-23,Sunday,43706510.0,4243.0,Jul,
3,Pass,Jul 24,2023-07-24,Monday,26105167.0,4243.0,Jul,
4,Pass,Jul 25,2023-07-25,Tuesday,26003569.0,4243.0,Jul,


Now that we have all the necessary functions to scrape the data, we will now need to write a function to upload the data onto our SQL server. Below is the code to upload the data onto our SQL server.

In [ ]:
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE=moviereleases;UID=dell;PWD=password"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = create_engine(connection_url)

def insert_to_db(df, tbl_name, eng):
    """df = dataframe, tbl_name for labelling purposes and the engine"""
    df.to_sql(tbl_name, con = eng, if_exists = 'append')#uploads to SQL
    return "Inserted " + " to "+tbl_name

## Almost done. Now we make the loop

Now that we have all the functions to scrape and upload the data, we now write the code to loop through it all.

In [ ]:
#urls we will be using
calend_url = 'https://www.boxofficemojo.com/calendar/{date}/'
base_url = 'https://www.boxofficemojo.com{movie_}'


# first for loop to loop through each month and the movie releases
for date, month in month_dict.items():
    m_url = calend_url.format(date=date)
    monthly_releases = month_release_processing(m_url, month)

    #upload the first table (monthly releases page) to SQL server
    insert_to_db(monthly_releases, 'monthlyreleases', engine)
    print("uploaded monthly releases for the month of "+date)

    #add a sleep function so we don't get our IP restricted
    time.sleep(3) 

    
    #get individual movie urls
    movie_urls = get_urls(monthly_releases)

    #second for loop to loop through the urls we get
    for movie in movie_urls:
        movie_url = base_url.format(movie_=movie['M_URL'])
        movie_db = movie_rev_processing(movie_url)

        movie_db['url'] = movie['M_URL']
        
        #add a sleep function so we don't get our IP restricted
        time.sleep(3)
        movie_name = get_movie_title(movie_url)
        movie_db['Movie'] = movie_name

        #upload to SQL server
        insert_to_db(movie_db, 'dailytable', engine)
        print("uploaded "+movie_name)
        
        #add a sleep function so we don't get our IP restricted
        time.sleep(3)

    print("finished "+date)

print("finished")

## We're almost done. We just need to do some clean up

Now that we have scraped all the data we needed, we have to clean up our table as their are duplicates present in the data. For some reason the website will publish the same movie in separate months, creating duplicates in our data. We will use SQL to clean it up. While you might be wondering, "Hey, why didn't we clean this earlier in the steps up above?", well my response is, I didn't know there were duplicates until after I ran my code. And writing another function to clean it up is not worth re-running and waiting a few hours to loop through all these movies again.

So what we will do is execute the following SQL code and it will remove our duplicatese


In [ ]:
-- ADDING UNIQUE ROW NUMBER

SELECT *,  ROW_NUMBER() over (order by Movie, Date) as row_num
INTO dev_table
FROM dbo.dailytable


--- SUBSTRING - ADDING MOVIE ID, MOVIE KEY, AND DUPLICATE INDICATOR COLUMNS

ALTER TABLE dbo.dev_table
ADD movie_id varchar(255),
ADD moviekey varchar(255),
ADD dupe_count INT

UPDATE dbo.dev_table
SET dbo.dev_table.movie_id = SUBSTRING(url,
		CHARINDEX('/release/',url) + len('/release/') ,
		charindex('/?ref', url) - len('/release/') - 1)
FROM dbo.dev_table

UPDATE dbo.dev_table
SET dbo.dev_table.moviekey = CONCAT(Date, Movie,movie_id)

-- IDENTIFY DUPLICATES 
SELECT 
		e.moviekey,
		e.row_num,
		T.rn
INTO temp
FROM moviereleases.dbo.dev_table e
	INNER JOIN
	(
	SELECT *,
	row_number() OVER (PARTITION BY moviekey order by row_num) rn
	FROM dbo.dev_table) T on e.moviekey = T.moviekey and e.row_num = T.row_num;

-- ADDING VALUES TO OUR DUPE COUNT COLUMN
UPDATE dbo.dev_table
SET dbo.dev_table.dupe_count = dbo.temp.rn
FROM dbo.dev_table LEFT JOIN dbo.temp
ON dbo.dev_table.row_num = dbo.temp.row_num

-- DELETING DUPLICATES
DELETE FROM dbo.dev_table
WHERE dupe_count > 1

-- TABLE WE CARE ABOUT
SELECT * FROM dev_table
WHERE Category = 'Pass'
ORDER BY Date, Movie